In [12]:
from itertools import product
import numpy as np
import random
import os
import shutil

In [13]:
def generate_components(gym_type):
    if gym_type == 'transformer':
        model_names = ['_'.join(list(_)) for _ in list(product(['TSGym'],
                                                            ['False'], # gym_x_mark
                                                            ['False', 'True'], # gym_series_sampling
                                                            ['None', 'Stat', 'RevIN', 'DishTS'], # gym_series_norm
                                                            ['None', 'MA', 'MoEMA', 'DFT'], # gym_series_decomp
                                                            ['False', 'True'], # gym_channel_independent
                                                            ['inverted-encoding', 'series-encoding', 'series-patching'], # gym_input_embed
                                                            ['Transformer'], # gym_network_architecture
                                                            ['self-attention', 'auto-correlation', 'sparse-attention', 'frequency-enhanced-attention', 'destationary-attention'], # gym_attn
                                                            ['null'], # gym_feature_attn， null
                                                            ['True'], # gym_encoder_only
                                                            ['False'], # gym_frozen
                                                            ['HP'],
                                                            ['64-256', '256-1024'], # d_model, d_ff
                                                            ['2', '3'], # encoder layers
                                                            ['10', '20', '50'], # training epochs
                                                            ['MAPE', 'MASE', 'SMAPE'], # loss functions
                                                            ['0.0001', '0.001'], # learning rate
                                                            ['null', 'type1'] # learning rate strategy
                                                            ))]
    elif gym_type == 'LLM':
        model_names = ['_'.join(list(_)) for _ in list(product(['TSGym'],
                                                            ['False'], # gym_x_mark
                                                            ['False'], # gym_series_sampling
                                                            ['None', 'Stat', 'RevIN', 'DishTS'], # gym_series_norm
                                                            ['None', 'MA', 'MoEMA', 'DFT'], # gym_series_decomp
                                                            ['True'], # gym_channel_independent
                                                            ['series-patching'], # gym_input_embed
                                                            ['LLM-GPT4TS', 'LLM-TimeLLM'], # gym_network_architecture
                                                            ['self-attention'], # gym_attn
                                                            ['null'], # gym_feature_attn
                                                            ['True'], # gym_encoder_only
                                                            ['False', 'True'], # gym_frozen
                                                            ['HP'],
                                                            ['64-256', '256-1024'], # d_model, d_ff
                                                            ['6'], # encoder layers
                                                            ['10', '20', '50'], # training epochs
                                                            ['MAPE', 'MASE', 'SMAPE'], # loss functions
                                                            ['0.0001', '0.001'], # learning rate
                                                            ['null', 'type1'] # learning rate strategy
                                                            ))]
    elif gym_type == 'TSFM':
        model_names = ['_'.join(list(_)) for _ in list(product(['TSGym'],
                                                            ['False'], # gym_x_mark
                                                            ['False'], # gym_series_sampling
                                                            ['None', 'Stat', 'RevIN', 'DishTS'], # gym_series_norm
                                                            ['None', 'MA', 'MoEMA', 'DFT'], # gym_series_decomp
                                                            ['True'], # gym_channel_independent
                                                            ['series-patching'], # gym_input_embed
                                                            ['TSFM-Timer', 'TSFM-Moment'], # gym_network_architecture
                                                            ['self-attention'], # gym_attn
                                                            ['null'], # gym_feature_attn
                                                            ['True'], # gym_encoder_only
                                                            ['False', 'True'], # gym_frozen
                                                            ['HP'],
                                                            ['64-256', '256-1024'], # d_model, d_ff
                                                            ['6'], # encoder layers
                                                            ['10', '20', '50'], # training epochs
                                                            ['MAPE', 'MASE', 'SMAPE'], # loss functions
                                                            ['0.0001', '0.001'], # learning rate
                                                            ['null', 'type1'] # learning rate strategy
                                                            ))]
    else:
        model_names = ['_'.join(list(_)) for _ in list(product(['TSGym'],
                                                        ['False'], # gym_x_mark
                                                        ['False', 'True'], # gym_series_sampling
                                                        ['None', 'Stat', 'RevIN', 'DishTS'], # gym_series_norm
                                                        ['None', 'MA', 'MoEMA', 'DFT'], # gym_series_decomp
                                                        ['False', 'True'], # gym_channel_independent
                                                        ['inverted-encoding', 'series-encoding', 'series-patching'], # gym_input_embed
                                                        ['MLP', 'GRU'], # gym_network_architecture
                                                        ['null'], # gym_attn
                                                        ['null'], # gym_feature_attn, null
                                                        ['True'], # gym_encoder_only
                                                        ['False'], # gym_frozen
                                                        ['HP'],
                                                        ['64-256', '256-1024'], # d_model, d_ff
                                                        ['2', '3'], # encoder layers
                                                        ['10', '20', '50'], # training epochs
                                                        ['MAPE', 'MASE', 'SMAPE'], # loss functions
                                                        ['0.0001', '0.001'], # learning rate
                                                        ['null', 'type1'] # learning rate strategy
                                                        ))]

    return model_names

In [14]:
for gym_type in ['transformer', 'non_transformer', 'LLM', 'TSFM']:
    random.seed(42)
    model_names = generate_components(gym_type)
    print(len(model_names))
    model_names = random.sample(model_names, 1000)
    for dataset in ['M4']:
        # 模板文件路径
        template_path = f'./scripts/short_term_forecast/TSGym_{dataset}.sh'
            
        # 输出目录
        output_dir = f'./scripts/short_term_forecast/gym_{gym_type}'

        # 确保输出目录存在
        if os.path.exists(output_dir):
            print('delete current folder!')
            shutil.rmtree(output_dir)
        os.makedirs(output_dir, exist_ok=True)

        # 读取模板内容
        with open(template_path, 'r') as file:
            template_content = file.read()

        # 对于每个模型名称，生成一个 shell 脚本
        for model_name in model_names:
            file_name = model_name

            HP = model_name[model_name.find('_HP')+4:]
            model_name = model_name[:model_name.find('_HP')]

            dm_df, el, epochs, loss, lr, lr_strategy = HP.split('_')
            dm, df = dm_df.split('-')[0], dm_df.split('-')[1]

            # 替换模型名称
            script_content = template_content.replace('$model_name', model_name)
            script_content = script_content.replace(f'$d_model', dm)
            script_content = script_content.replace(f'$d_ff', df)
            script_content = script_content.replace(f'$e_layers', el)
            script_content = script_content.replace(f'$train_epochs', epochs)
            script_content = script_content.replace(f'$loss', loss)
            script_content = script_content.replace(f'$learning_rate', lr)
            script_content = script_content.replace(f'$lradj', lr_strategy)
            
            # 定义输出文件名
            output_file = os.path.join(output_dir, f'{file_name}.sh')
            
            # 写入新的 shell 脚本
            with open(output_file, 'w') as file:
                file.write(script_content)

            # print(f'Generated {output_file}')

552960
delete current folder!
221184
delete current folder!
4608
delete current folder!
4608
delete current folder!
